In [1]:
#导入常用的 Python 库和模块
from selenium import webdriver#导入 Selenium 库中的 webdriver 模块。Selenium 是一个流行的 Web 自动化测试工具，用于通过浏览器进行自动化操作，比如模拟用户行为（点击、输入、滚动等）。
from selenium.webdriver.common.by import By#导入 Selenium 库中的 By 模块。By 是 Selenium 用来定位页面元素的方式。
from selenium.webdriver.support.ui import WebDriverWait#导入 Selenium 库中的 WebDriverWait 模块。WebDriverWait 是一个等待机制，用于在自动化测试中等待元素的出现或满足特定条件
from selenium.webdriver.support import expected_conditions as EC#导入 Selenium 库中的 expected_conditions (简称 EC) 模块。
import time
import random
import csv
import pandas as pd
import os

# 定义输入和输出CSV文件路径
input_file_path = os.path.join(os.getcwd(), '视频列表4.1余.xlsx')  # 输入视频链接的文件
output_file_path = os.path.join(os.getcwd(), 'youtube_comments(4.1).csv')  # 输出评论的文件

# 定义表头
headers = ['Video Link', 'User ID', 'Comment', 'Likes']
if not os.path.exists(output_file_path) or os.path.getsize(output_file_path) == 0:
    with open(output_file_path, mode='w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow(headers)  # 写入表头


In [2]:
# 读取视频链接-xlsx
df = pd.read_excel('视频列表4.1余.xlsx')  # 使用 pandas 读取 Excel 文件
# 打印列名以确认
print("列名:", df.columns.tolist())

# 替换 '实际列名' 为你在打印输出中看到的正确列名
video_links = df['Video URL'].dropna().tolist()  # 确保替换为正确的列名

# 输出视频链接以确认
print("视频链接:", video_links)

列名: ['Video URL', 'Video Title', 'Youtuber', 'Publish Date', 'View Count', 'Video ID']
视频链接: ['https://www.youtube.com/watch?v=rMVu0CNrJmU&pp=ygUVMTQ0LWhvdXIgdHJhbnNpdCB2aXNh']


In [ ]:
# 初始化浏览器
options = webdriver.ChromeOptions()#通过 ChromeOptions 可以设置浏览器启动时的参数，例如语言设置、用户代理（User-Agent）等。
options.add_argument("--lang=en-US")#向 Chrome 浏览器添加了一个启动参数，设置浏览器的语言为 英语（美国）。
options.add_argument(
    "user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36")#这行代码为浏览器设置了一个 自定义的用户代理（User-Agent） 字符串。用户代理是一种告诉服务器客户端信息（例如操作系统、浏览器版本等）的方式。
driver = webdriver.Chrome(options=options)#创建了一个 Chrome 浏览器实例，并将先前设置的选项 options 应用到浏览器中。
wait = WebDriverWait(driver, 6)#创建了一个 显式等待（Explicit Wait） 对象，名为 wait
# 爬取url里的视频评论
def smooth_scroll(driver, scroll_amount=300, steps = 2,processed_buttons=None):#定义了一个名为 smooth_scroll 的函数，功能是模拟平滑滚动页面的操作，并检查页面中是否加载了评论。
    """平滑滚动并检查评论加载状态"""
    if processed_buttons is None:
        processed_buttons = set()#如果函数没有传入 processed_buttons 集合（即该参数为 None），则初始化一个空集合。这是为了避免在函数执行过程中多次传递 None 值。
        
    current_position = driver.execute_script("return window.pageYOffset;")#这行代码使用 JavaScript 脚本获取当前页面的垂直滚动位置（即页面从顶部开始滚动的距离），并将其存储在 current_position 变量中。
    
    # 获取当前窗口高度并用于计算可见区域
    window_height = driver.execute_script("return window.innerHeight;")#这行代码通过 JavaScript 获取当前浏览器窗口的高度（可见区域的高度），并将其存储在 window_height 变量中。
    target_position = current_position + scroll_amount#这行代码计算了目标滚动位置，即当前滚动位置加上 scroll_amount（每次滚动的距离）。这个目标位置表示在本次滚动后，页面应该达到的位置。
    visible_area_bottom = current_position + window_height#这行代码计算了浏览器窗口底部的位置（可见区域的底部）。它是当前滚动位置加上窗口的高度，表示当前可见区域的底部边界。
    
    # 添加随机化滚动距离
    scroll_amount = random.uniform(400, 600)#生成一个在 300 到 600 之间的随机浮动滚动距离，使得每次滚动的距离都有一定的随机性，从而模拟更加自然的滚动行为。
    
    # 在滚动循环添加"展开评论回复"尝试
    try:
        # 获取当前滚动位置
        scroll_position = driver.execute_script("return window.pageYOffset;")#JavaScript 获取当前的滚动位置 pageYOffset，即页面顶部到当前可见区域顶部的距离。这个值用来判断哪些按钮在当前可视区域内。
        # 只获取当前可见区域及其下方的按钮
        buttons = driver.find_elements(By.XPATH, "//ytd-button-renderer/yt-button-shape/button/yt-touch-feedback-shape/div")#通过 XPath 找到所有可能的“展开回复”按钮。这些按钮在 YouTube 页面上通常用于展开评论区的回复内容。find_elements 会返回页面上所有匹配该 XPath 的元素。
        
        for button in buttons:#for button in buttons: 遍历所有找到的按钮元素，检查每个按钮是否在当前页面可见区域内。
            try:
                # 获取按钮的位置
                button_position = button.location['y']#使用 button.location['y'] 获取按钮的垂直位置，表示按钮顶部距离页面顶部的像素数。
                
                # 只处理在当前可见区域和目标位置之间的按钮
                if button_position < scroll_position or button_position > target_position:#如果按钮的垂直位置在当前滚动位置之前，或者超过目标位置，则跳过当前按钮。这样可以避免处理滚动后不在可视区域的按钮。
                    continue
                    
                # 判断按钮是否在可见区域内
                if button_position >= current_position and button_position <= visible_area_bottom:
                    parent_button = button.find_element(By.XPATH, "./ancestor::button")#通过 XPath 找到按钮的父元素。这里假设父元素是 <button>，并通过 find_element 获取它。
                    button_text = parent_button.get_attribute("aria-label") or ''
                    # 创建按钮的唯一标识符
                    button_id = f"{button_text}_{button_position}"#这里通过 按钮文本 和 按钮位置 创建了一个唯一的按钮 ID，用于标识已经处理过的按钮。这是为了避免重复点击已经处理过的按钮。
                    
                    # 检查是否已处理过该按钮
                    if ("replies" in button_text and 
                        button_id not in processed_buttons and 
                        parent_button.is_displayed()):
                        try:
                            # 点击按钮，展开评论回复
                            parent_button.click() 
                            processed_buttons.add(button_id)  # 将按钮 ID 添加到 processed_buttons 集合中，表示该按钮已经处理过，避免重复点击
                            # 使用显式等待来代替time.sleep，等待评论加载完成
                            WebDriverWait(driver, 7).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, 'ytd-comment-thread-renderer')))
                            # 增加更多时间来确保评论区完全展开
                            time.sleep(random.uniform(4, 6))  # 等待评论加载完成后再继续操作
                        # 处理完评论后，继续向下滑动页面
                            driver.execute_script("window.scrollBy(0, 400);")
                            time.sleep(random.uniform(1, 3))  # 随机等待时间，避免行为被识别为爬虫
                        except Exception as e:
                            print(f"点击展开回复按钮时出错: {e}")
                            continue
            except Exception as e:
                continue
                
    except Exception as e:
        print(f"展开回复时出错: {e}")
    
    # 分步滚动
    steps = 1
    for step in range(steps):
        current_scroll_position = current_position + scroll_amount #计算滚动后的目标位置，即：当前滚动位置（current_position）加上每次滚动的距离（scroll_amount）。
        driver.execute_script(f"window.scrollTo({current_position}, {current_scroll_position});")#通过 driver.execute_script() 执行 JavaScript，调用 window.scrollTo() 来滚动页面。
        current_position = current_scroll_position#在每次滚动后，更新 current_position 为当前的滚动位置，以便在下一次滚动时使用
        # 等待页面加载评论内容
        WebDriverWait(driver, 5).until(
            EC.presence_of_all_elements_located((By.CSS_SELECTOR, "#content-text"))
        )#使用显示等待减少不必要的等待时间
        # 检查页面是否已加载更多内容，使用页面高度变化来判断
        prev_height = driver.execute_script("return document.body.scrollHeight;")
        
        current_height = driver.execute_script("return document.body.scrollHeight;")
        if current_height > prev_height:
            # 页面内容增加了，表示评论已经加载
            print("评论已加载，继续滚动")
        else:
            print("评论加载中...")
            time.sleep(random.uniform(2, 3))  # 等待更多时间来确保评论加载

    return current_position

failed_links=[]
# 遍历视频链接并抓取评论
for link in video_links:
    try:
        if not driver or not driver.window_handles:  # 检查浏览器是否还在运行
            driver = webdriver.Chrome(options=options)  # 如果浏览器关闭了就重新打开
            wait = WebDriverWait(driver, 10)
            
        driver.get(link)
        time.sleep(3)
        print(f"开始抓取视频: {link}")
        
        driver.execute_script("window.scrollBy(0, 400);")
        time.sleep(2)
        
        # 定位评论数量元素
        wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "h2#count span.style-scope.yt-formatted-string:first-child")))#使用 WebDriverWait 等待评论数的元素加载（h2#count span.style-scope.yt-formatted-string:first-child），然后抓取评论数量的文本内容。
        comment_count_element = driver.find_element(By.CSS_SELECTOR, "h2#count span.style-scope.yt-formatted-string:first-child")
        comment_count_text = comment_count_element.text.strip()
        
        # 解析评论数量
        comment_count = 0#comment_count 被初始化为 0，以便在后续的代码中用于存储最终的评论数量。
        comment_count_text = comment_count_text.replace(",", "").strip()#将评论数量中的逗号去掉；去除字符串两端的多余空格，确保处理干净的文本内容
        if "万" in comment_count_text:
            comment_count = int(float(comment_count_text.replace("万", "")) * 10000)#如果评论数量中包含“万”字（例如 "5万"），则将该单位去除并将其转换为数字（float(comment_count_text.replace("万", ""))），然后乘以 10,000（* 10000），将“万”单位转换为真实的评论数量。
        elif "K" in comment_count_text:
            comment_count = int(float(comment_count_text.replace("K", "")) * 1000)#如果评论数量中包含“K”字母（例如 "1.5K"），则将该单位去除并将其转换为数字，乘以 1000（* 1000），将“K”单位转换为真实的评论数量。
        elif "M" in comment_count_text:
            comment_count = int(float(comment_count_text.replace("M", "")) * 1000000)#如果评论数量中包含“M”字母（例如 "2.3M"），则将该单位去除并将其转换为数字，乘以 1,000,000（* 1000000），将“M”单位转换为真实的评论数量。
        elif comment_count_text.isdigit():
            comment_count = int(comment_count_text)# 如果 comment_count_text 是纯数字（没有单位，如 12345），则直接将其转换为整数。
        
        # 根据评论数量动态设置抓取目标
        if comment_count > 2000:
            total_comments = 1000
        elif 1000 < comment_count <= 2000:
            total_comments = 800
        elif 500 < comment_count <= 1000:
            total_comments = 500
        else:
            total_comments = comment_count

        print(f"视频 {link} 的评论总数: {comment_count}，目标抓取 {total_comments} 条评论。")

        # 滑动到评论部分触发加载
        comment_section = driver.find_element(By.CSS_SELECTOR, "#comments")#通过 CSS 选择器定位页面中包含评论内容的元素。
        driver.execute_script("arguments[0].scrollIntoView();", comment_section)#scrollIntoView() 方法会将页面自动滚动，直到 comment_section（评论区域）完全出现在浏览器视窗中。这样做的目的是触发评论的加载，确保评论区的内容被加载并显示在页面上。
        time.sleep(2)
        
        # 初始化变量
        loaded_comments = 0
        scroll_attempts = 0
        max_scroll_attempts = 10
        comment_data = []
        processed_buttons = set()  # 用于存储所有已处理的按钮

        # 在while循环中使用新的滚动逻辑
        while loaded_comments < total_comments and scroll_attempts < max_scroll_attempts:#如果加载的评论数已经达到或超过目标评论数，或者滚动尝试次数超过最大值，循环将终止。
            is_loading = smooth_scroll(driver, scroll_amount=500, pause_time=0.3, processed_buttons=processed_buttons)
 
            # 显式等待，确保页面已经加载新的评论
            WebDriverWait(driver, 5).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "#content-text")))#WebDriverWait 是一个显式等待，它会等待直到页面上加载出所有评论元素 (#content-text 的 CSS 选择器代表评论文本)。
            ## 获取当前所有评论
            comments = driver.find_elements(By.CSS_SELECTOR, "#content-text")# 用于获取所有评论内容。#content-text 是每个评论的 CSS 选择器，因此该代码会获取页面上所有显示的评论元素。

            if len(comments) > loaded_comments:#判断页面中当前加载的评论数 len(comments) 是否大于之前已经加载的评论数 loaded_comments。如果是，表示页面加载了新的评论，则更新 loaded_comments 为当前评论的数；new_comments_count 计算新加载的评论数，并输出日志。
                new_comments_count = len(comments) - loaded_comments
                loaded_comments = len(comments)
                scroll_attempts = 0
                print(f"新加载评论数: {new_comments_count}，已加载总评论数: {loaded_comments}/{total_comments}")
                time.sleep(2) # 增加一个等待评论加载的时间，让新评论能被selenium反应
            else:
                scroll_attempts += 1#如果没有加载到更多的评论（即 len(comments) 没有增加），则增加 scroll_attempts，表示尝试次数增加。
                print(f"未检测到更多评论，尝试次数: {scroll_attempts}/{max_scroll_attempts}")
                
                if scroll_attempts > 3:#如果尝试次数超过 3 次，则执行一个向上滚动页面的操作；向上滚动的目的是让评论区的加载操作再次触发，可能是为了加载更多的评论或刷新加载状态。
                    driver.execute_script("window.scrollBy(0, -200);")
                    time.sleep(2)

        # 抓取评论内容
        print("抓取评论内容...")
        comments = driver.find_elements(By.CSS_SELECTOR, "#content-text")#使用 driver.find_elements 方法来查找页面上所有的评论内容
        user_ids = driver.find_elements(By.CSS_SELECTOR, "a#author-text span")#这行代码用来抓取评论者的用户名或用户ID
        likes = driver.find_elements(By.CSS_SELECTOR, "span#vote-count-middle")#这行代码用来抓取每条评论的点赞数。

        # 处理每条评论处理抓取到的评论内容，并对每条评论的点赞数进行处理和转换
        for i in range(min(len(comments), total_comments)):#取评论数 len(comments) 和希望抓取的评论总数 total_comments 中较小的值，确保不会超出评论的实际数量
            comment_text = comments[i].text.strip()#这一行获取每条评论的文本内容，并使用 .strip() 方法去除前后的空白字符。
            user_id = user_ids[i].text.strip() if i < len(user_ids) else "Unknown"#这一行获取评论作者的用户名。如果 i 小于 user_ids 列表的长度，说明该评论有作者信息，使用 .text.strip() 获取并去除前后的空白字符；如果 i 超过了 user_ids 列表的长度（即没有作者信息），则将 user_id 设置为 "Unknown"。
            like_text = likes[i].text.strip() if i < len(likes) else "0"#这一行获取评论的点赞数
            
            # 处理点赞数:将这些格式统一转换为数字
            like_count = 0
            if "万" in like_text:
                like_count = int(float(like_text.replace("万", "")) * 10000)
            elif "K" in like_text:
                like_count = int(float(like_text.replace("K", "")) * 1000)
            elif "M" in like_text:
                like_count = int(float(like_text.replace("M", "")) * 1000000)
            elif like_text.isdigit():
                like_count = int(like_text)

            comment_data.append([link, user_id, comment_text, like_count])
        
        # 写入CSV文件
        with open(output_file_path, mode='a', newline='', encoding='utf-8') as file:
            writer = csv.writer(file)
            writer.writerows(comment_data)
        print(f"视频 {link} 的评论数据已写入，抓取到 {len(comment_data)} 条评论。")
        
    except Exception as e:
        print(f"视频 '{link}' 抓取失败: {e}")
        failed_links.append(link)
   
        try:
            driver.quit()  # 尝试关闭现有的浏览器
            #所有数据写入文件后，输出失败的链接
            if failed_links:
                print("以下视频的评论抓取失败：")
            for failed_link in failed_links:
                print(failed_link)
        except:
            pass
        driver = webdriver.Chrome(options=options)  # 重新打开浏览器
        wait = WebDriverWait(driver, 10)
        continue
        
# 关闭浏览器
driver.quit()
print(f"所有数据已写入文件：{output_file_path}")

There was an error managing chromedriver (error sending request for url (https://googlechromelabs.github.io/chrome-for-testing/known-good-versions-with-downloads.json)); using driver found in the cache


开始抓取视频: https://www.youtube.com/watch?v=rMVu0CNrJmU&pp=ygUVMTQ0LWhvdXIgdHJhbnNpdCB2aXNh
视频 https://www.youtube.com/watch?v=rMVu0CNrJmU&pp=ygUVMTQ0LWhvdXIgdHJhbnNpdCB2aXNh 的评论总数: 1242，目标抓取 800 条评论。
评论加载中...
新加载评论数: 28，已加载总评论数: 28/800
评论加载中...
未检测到更多评论，尝试次数: 1/10
评论加载中...
新加载评论数: 37，已加载总评论数: 65/800
评论加载中...
未检测到更多评论，尝试次数: 1/10
评论加载中...
未检测到更多评论，尝试次数: 2/10
评论加载中...
未检测到更多评论，尝试次数: 3/10
评论加载中...
未检测到更多评论，尝试次数: 4/10
评论加载中...
未检测到更多评论，尝试次数: 5/10
评论加载中...
未检测到更多评论，尝试次数: 6/10
评论加载中...
未检测到更多评论，尝试次数: 7/10
评论加载中...
未检测到更多评论，尝试次数: 8/10
评论加载中...
新加载评论数: 30，已加载总评论数: 95/800
评论加载中...
未检测到更多评论，尝试次数: 1/10
评论加载中...
未检测到更多评论，尝试次数: 2/10
评论加载中...
未检测到更多评论，尝试次数: 3/10
评论加载中...
未检测到更多评论，尝试次数: 4/10
评论加载中...
未检测到更多评论，尝试次数: 5/10
评论加载中...
未检测到更多评论，尝试次数: 6/10
评论加载中...
新加载评论数: 7，已加载总评论数: 102/800
评论加载中...
未检测到更多评论，尝试次数: 1/10
评论加载中...
新加载评论数: 4，已加载总评论数: 106/800
评论加载中...
新加载评论数: 7，已加载总评论数: 113/800
评论加载中...
新加载评论数: 2，已加载总评论数: 115/800
评论加载中...
新加载评论数: 5，已加载总评论数: 120/800
评论加载中...
新加载评论数: 2，已加载总评论数: 122/800
评论加载中...
新加载评论数